In [1]:
import os, random, sys, time, shutil, re, csv, datetime, pickle, math 

import pandas as pd

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.webdriver import Options

from bs4 import BeautifulSoup

In [2]:
chrome_options = Options()
#chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--log-level=3")  # fatal
chrome_options.add_argument("--start-maximized")

In [3]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

browser = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

#browser = webdriver.Chrome("chromedriver.exe", options=chrome_options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 88.0.4324
[WDM] - Get LATEST driver version for 88.0.4324


[WDM] - Driver [/home/carlos.cayres/.wdm/drivers/chromedriver/linux64/88.0.4324.96/chromedriver] found in cache


In [ ]:
# browser.get("https://clutch.co/it-services/outsourcing")
browser.get("https://clutch.co/profile/pattern-match")

In [ ]:
# image = browser.find_element_by_class_name('company_logotype') #class="company_logotype"
# image.click()
soup = BeautifulSoup(browser.page_source, features='html.parser')
url_list = soup.find_all("div",attrs={"class" : "company_logotype"}, recursive=True)

url_list

In [ ]:
for url in url_list:
    print(url.find('a')['href'])
    #browser.get(f"https://clutch.co{url.find('a')['href']}")
    time.sleep(3)
    
    soup = BeautifulSoup(browser.page_source, features='html.parser')

    nome = soup.find("h1", attrs={"class" : "header-company--title"})
    nome = nome.text.replace('\n', '').strip()

    nota = soup.find("span", attrs={"itemprop" : "ratingValue"})
    nota = nota.text.replace('\n', '').strip()

    local = soup.find("span", attrs={"class" : "location-name"})
    local = local.text.replace('\n', '').strip()
    
    clients = ''
    try:
        clients = soup.find("div", attrs={"class" : "field-name-clients"})
        clients =  ' '.join([p.text.replace('\n', ' ').strip() for p in clients.find_all('p')])
    except:
        try:
            clients = soup.find("div", attrs={"class" : "field c"})
            clients = ' '.join([p.text.replace('\n', ' ').strip() for p in clients.find_all('p')])
            #clients = clients.find('p').text.replace('\n', ' ').strip()
        except:
            print(f"CLIENTES NÃO ENCONTRADOS EM {nome}")
            
    print(f"nome: {nome}")
    print(f"nota: {nota}")
    print(f"local: {local}")
    print(f"clientes: {clients}")

In [ ]:
for i in range(4):
    browser.get(f"https://clutch.co/it-services/outsourcing?page={i}")
    soup = BeautifulSoup(browser.page_source, features='html.parser')
    url_list = soup.find_all("div",attrs={"class" : "company_logotype"}, recursive=True)
    for url in url_list:
        print(url.find('a')['href'])

In [ ]:
# nome -- h1 class header-company--title 
# nota -- span itemprop ratingValue
# local -- span class location-name
# clients -- div class field-name-clients // field c -> p
browser.get("https://clutch.co/profile/pattern-match")

In [ ]:
soup = BeautifulSoup(browser.page_source, features='html.parser')

nome = soup.find("h1", attrs={"class" : "header-company--title"})
nome = nome.text.replace('\n', '').strip()

nota = soup.find("span", attrs={"itemprop" : "ratingValue"})
nota = nota.text.replace('\n', '').strip()

local = soup.find("span", attrs={"class" : "location-name"})
local = local.text.replace('\n', '').strip()

try:
    clients = soup.find("div", attrs={"class" : "field-name-clients"})
    clients = ' '.join([p.text for p in clients.find_all('p')])
except:
    try:
        clients = soup.find("div", attrs={"class" : "field c"})
        clients = ' '.join([p.text for p in clients.find_all('p')])
    except:
        print(nome)

print(f"nome: {nome}")
print(f"nota: {nota}")
print(f"local: {local}")
print(f"clientes: {clients}")

In [ ]:
soup = BeautifulSoup(browser.page_source, features='html.parser')


main_div = soup.find("div", attrs={"class" : "module-list"}) 
main_div.find_all("span")

valor = ''
preco_hora = ''
funcionarios = ''

main_div = [i.text for i in main_div.find_all("span")]
main_div
valor, preco_hora, funcionarios = [main_div[i] for i in (0, 1, 2)]

print(f"valor {valor}")
print(f"preco_hora {preco_hora}")
print(f"funcionarios {funcionarios}")